In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [2]:
data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [71]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [5]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. 
    """
    
    items = items_weights['item_id']
    weights = items_weights['weights']
    recs = np.random.choice(items, size=n, replace=False, p=weights)
    
    return recs.tolist()

In [6]:
items_weights = data_train.groupby('item_id')['sales_value'].sum().reset_index()
items_weights['weights'] = items_weights['sales_value'] / items_weights['sales_value'].sum()
items_weights.drop(columns=['sales_value'], axis=1, inplace=True)
items_weights.weights.sum()

0.9999999999999999

In [72]:
%%time

result['random_recommendation'] = result['user_id'].apply(lambda x: 
                                                          weighted_random_recommendation(items_weights, n=5))
result.head(2)

CPU times: user 2.81 s, sys: 276 ms, total: 3.09 s
Wall time: 3.55 s


,user_id,actual,random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[9676866, 1052294, 12810556, 955765, 883963]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[6534178, 1069637, 1053628, 8118665, 9526574]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [87]:
result = pd.read_csv('predictions_basic.csv')
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [151]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)  # Тут нет [:k] !!
    recommended_list = np.array(recommended_list[:k])
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [107]:
import re

In [147]:
def get_list(string):
    return re.findall(r'[^[ \\]\S\d*', string)

In [169]:
headers = result.columns[2:].tolist()
precisions = []
for model in headers:
    precisions.append(result.apply(lambda row: 
             precision_at_k(get_list(row[model]), get_list(row['actual'])), axis=1).mean())

In [176]:
pd.DataFrame(precisions, index=headers)

,0
random_recommendation,0.000588
popular_recommendation,0.155240
itemitem,0.033595
cosine,0.035260
tfidf,0.036141
own_purchases,0.179987
